# Gemini API intro

In [ ]:
from google import genai

# looks automatically after the key
# one of GOOGLE_API_KEY and GEMINI_API_KEY
client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Explain how AI works in a few words",
)

print(response.text)

AI learns patterns from data to make decisions and perform tasks.


In [2]:

def ask_gemini(prompt, model = "gemini-2.5-flash"):
    response = client.models.generate_content(
        model=model,
        contents=prompt,
    )

    return response

response = ask_gemini("Give me 5 some data engineering jokes, structure it in short points")
print(response.text)

Here are 5 data engineering jokes, structured in short points:

1.  A data engineer's job is 10% coding, 90% debugging why the data from the 'clean' source looks like it was generated by a toddler with a keyboard.
2.  What's a data engineer's biggest fear? A fully automated data pipeline... that actually works for more than a day.
3.  Heard a data engineer say they finished their project. I almost choked on my coffee – that's like saying you've finished eating a never-ending buffet.
4.  A data scientist walks into a bar and asks for a perfectly clean, curated dataset. The bartender says, "Sorry, we only serve real-world data here."
5.  My data pipeline isn't broken, it's just... *exercising its right to unpredictability*.


In [ ]:
from pydantic import BaseModel

# knows that GenerateContentResponse is a pydantic model
# -> we can work with it in a OOP manner
isinstance(response, BaseModel)

True

In [10]:
dict(response).keys()

dict_keys(['sdk_http_response', 'candidates', 'create_time', 'model_version', 'prompt_feedback', 'response_id', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])

In [12]:
response.model_version

'gemini-2.5-flash'

In [14]:
response.sdk_http_response

HttpResponse(
  headers=<dict len=11>
)

In [16]:
response.candidates

[Candidate(
   content=Content(
     parts=[
       Part(
         text="""Here are 5 data engineering jokes, structured in short points:
 
 1.  A data engineer's job is 10% coding, 90% debugging why the data from the 'clean' source looks like it was generated by a toddler with a keyboard.
 2.  What's a data engineer's biggest fear? A fully automated data pipeline... that actually works for more than a day.
 3.  Heard a data engineer say they finished their project. I almost choked on my coffee – that's like saying you've finished eating a never-ending buffet.
 4.  A data scientist walks into a bar and asks for a perfectly clean, curated dataset. The bartender says, "Sorry, we only serve real-world data here."
 5.  My data pipeline isn't broken, it's just... *exercising its right to unpredictability*."""
       ),
     ],
     role='model'
   ),
   finish_reason=<FinishReason.STOP: 'STOP'>,
   index=0
 )]

## Tokens

In [18]:
response.usage_metadata

GenerateContentResponseUsageMetadata(
  candidates_token_count=187,
  prompt_token_count=15,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=15
    ),
  ],
  thoughts_token_count=1197,
  total_token_count=1399
)